# Install Libraries

In [ ]:
#conda install theano

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import scipy
from statistics import stdev
from scipy import stats

from sklearn.metrics import mean_squared_error

from sklearn.decomposition import PCA


# Custom Libraries for saving excel and transformation of variables
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# from custom_mmm_modelling_v1_updated import MMM_Modelling
from custom_mmm_modelling import MMM_Modelling

In [ ]:
# To display numbers in decimal separated format
pd.options.display.float_format = '{:20,.5f}'.format
pd.options.display.max_columns = 99

# User Defined Functions

In [ ]:
import statsmodels
import statsmodels.api as sm
import numpy as np
# Set the link function to identity
statsmodels.genmod.families.links.identity()
def glm_lr(x,y):
    OLS_from_GLM = sm.GLM(y, sm.add_constant(x))
    #OLS_from_GLM = sm.GLM(y_norm, sm.add_constant(dep))
    #R = [1.0] * (len(all_independent_variables_new)+1)
    #result3 = OLS_from_GLM.fit_constrained((R, y.max()))
    lrfit = OLS_from_GLM.fit()
    return lrfit
def get_1d_x(x_train):
    pca = PCA(n_components=1)
    pca.fit(x_train)
    x = pca.transform(x_train)
    return x
def get_r2(y, yhat):
    SS_Residual = sum((y-yhat)**2)
    SS_Total = sum((y-np.mean(y))**2)
    r2 = 1 - ((float(SS_Residual))/SS_Total)
    return r2
def sm_ols(x,y):
    result = sm.OLS(y, sm.add_constant(x)).fit()
    return result
def new_media(command):
    a = []
    split1 = command.split('_')
    for i in split1:
        for j in media_variables:
            split2 = j.split('_')
            if int(split2[-1]) == int(i):
                a.append(j)           
    return a

In [ ]:
def get_lines_value_tuple(key, value, agg="mean"):
    
    key_name = key.split("__")
    
    return (key_name[0], 
            {f"{key_name[0]}_dim_0": int(key_name[1])} if len(key_name) > 1 else {}, 
            [value[agg]]) 

In [ ]:
def plot_trace(trace):
    # Traceplot with vertical lines at the mean value
    
    matplotlib.style.use('seaborn')
    

    ax = pm.traceplot(trace, figsize=(12, len(trace.varnames)*2.2),
                      lines=[get_lines_value_tuple(k, v, "mean")  for k, v in pm.summary(trace).iterrows()],
                           )
    
    #ax[0,0].set_prop_cycle(cycler('color', ['c', 'm', 'y', 'k']))
    #ax[1,0].set_prop_cycle(cycler('color', ['c', 'm', 'y', 'k']))
    
    matplotlib.rcParams['font.size'] = 8
    
    # Labels with the median value
    for i, mn in enumerate(pm.summary(trace)['mean']):
        ax[i, 0].annotate('{:0.2f}'.format(mn), xy = (mn, 0), xycoords = 'data', size = 8,
                          xytext = (-18, 18), textcoords = 'offset points', rotation = 90,
                          va = 'bottom', fontsize = 'large', color = 'red')

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def get_1d_x(x_train):
    pca = PCA(n_components=1)
    pca.fit(x_train)
    x = pca.transform(x_train)
    return x

In [ ]:
def get_yhat(x_unscaled, coefs,intercept):
    
    y_hat = (x_unscaled * coefs).sum(axis=1) + intercept
    
    return y_hat
def get_diagnostics(y, yhat, X):
    # Calculate r2, rmse and log-liklihood values
    #yhat = model.predict(X)    
    
    SS_Residual = sum((y-yhat)**2)
    SS_Total = sum((y-np.mean(y))**2)
    r2 = 1 - ((float(SS_Residual))/SS_Total)
    adjusted_r_squared = 1 - ((1-r2)*(len(y)-1)/(len(y)-X.shape[1]-1-1))
    
    
    mse = float(SS_Residual)/len(y)
    rmse = np.sqrt(mse)
    
    log_likelihood = np.log(mse)
    
    aic = - len(y) * log_likelihood + 2 * (X.shape[1]-1)
    
    bic = - 2 * log_likelihood + (X.shape[1]-1) * np.log(len(y))
    
    mape = abs((yhat/y)-1).mean()*100
    diag = pd.DataFrame()
    diag['diag'] = ['r2', 'adjusted_r_squared', 'rmse', 'mse', 'log_likelihood', 'aic', 'bic', 'mape']
    diag['value'] = [r2, adjusted_r_squared, rmse, mse, log_likelihood, aic, bic, mape]
    return diag

### Please select you need scaled data or unscale data


In [ ]:

select_df = pd.DataFrame({'Scaled_data':[False], 'Unscale_Data':[True]}) 

In [ ]:
def mean_scale(train, val):
    
 
        
    if (select_df.Scaled_data[0]==True):
         scaler=train.max() - train.min()
    if (select_df.Unscale_Data[0]==True):
        scaler = 1 
    
    train_scaled = train/scaler
    val_scaled = val/scaler
    
    return scaler, train_scaled, val_scaled

In [ ]:
def split_scale_order(df, date_col, sales_col,
                      start_date_train, end_date_train,
                      start_date_val, end_date_val):
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    df = df.sort_index()
    
    train = df.loc[start_date_train:end_date_train]
    val = df.loc[start_date_val:end_date_val]
    
    scaler, train_scaled, val_scaled = mean_scale(train, val)
    
    x_train = train_scaled.copy()
    y_train = x_train.pop(sales_col)
    
    cols = x_train.columns
    x_train['intercept'] = 1
    col_list = ['intercept']
    col_list.extend(list(cols))
    x_train = x_train[col_list]

    x_val = val_scaled.copy()
    y_val = x_val.pop(sales_col)
    x_val['intercept'] = 1
    x_val = x_val[col_list]
    
    return scaler, x_train, y_train, x_val, y_val

In [ ]:
def get_errors(trace, x, y, lr):
    
    df_samples = get_samples(trace)
    lr_coefs = np.insert(lr.coef_[1:], 0, lr.intercept_, axis=0)
    
    true_regression_line = (x * lr_coefs).sum(axis=1).values
    mean_bayes_line = (x * pm.summary(trace)['mean'].values).sum(axis=1).values
    map_bayes_line = (x * df_samples.mode().iloc[0].values).sum(axis=1).values
    
    rmse_lr = np.sqrt(mean_squared_error(y.values, true_regression_line))
    rmse_mean = np.sqrt(mean_squared_error(y.values, mean_bayes_line))
    rmse_map = np.sqrt(mean_squared_error(y.values, map_bayes_line))
    mape_lr = mean_absolute_percentage_error(y.values, true_regression_line)
    mape_mean = mean_absolute_percentage_error(y.values, mean_bayes_line)
    mape_map = mean_absolute_percentage_error(y.values, map_bayes_line)
    
    return rmse_lr, rmse_mean, rmse_map, mape_mean, mape_map, mape_lr, \
           true_regression_line, mean_bayes_line, map_bayes_line

In [ ]:
def get_samples(trace):
    samples=[]
    for t in trace:
        row = {}
        row = {'intercept': t['Intercept'], 
               'beta0': trace[0]['beta0'], 
               'beta1': trace[0]['beta1'], 
               'beta2': trace[0]['beta2'], 
               'beta3': trace[0]['beta3'], 
               'beta4': trace[0]['beta4'],
               'beta5': trace[0]['beta5'], 
               'beta6': trace[0]['beta6'], 
               'beta7': trace[0]['beta7'], 
               'beta8': trace[0]['beta8'],
               'beta9': trace[0]['beta9'], 
               'beta10': trace[0]['beta10'], 
               'beta11': trace[0]['beta11'], 
               'beta12': trace[0]['beta12'], 
               'beta13': trace[0]['beta13'], 
               'beta14': trace[0]['beta14'],
             
#              'beta3': trace[0]['beta3'], 
#              'beta4': trace[0]['beta4']
              }
        samples.append(row)
    df_samples = pd.DataFrame(samples)
    return df_samples

In [ ]:
def linear_regression(X, y):
    # Equation for linear regression coefficients
    beta = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), y)
    return beta

def sk_lr(X, y):
    
    lr = LinearRegression()
    lr.fit(X, y)
    return lr

In [ ]:
def get_regression_plot(x, true_regression_line, mean_bayes_line, map_bayes_line, y):
    plt.figure(figsize=(12, 9))
    #plt.scatter(x, y.values, label='data', marker='x', c='b', alpha=0.3)
    xt, yt = zip(*sorted(zip(x, true_regression_line)))
    xb, yb = zip(*sorted(zip(x, mean_bayes_line)))
    xm, ym = zip(*sorted(zip(x, map_bayes_line)))
    plt.plot(xt, yt, label='true regression line', lw=1., c='y')
    plt.plot(xb, yb, label='Mean Bayesian Line', lw=1., c='g')
    plt.plot(xm, ym, label='MAP Bayesian Line', lw=1., c='r')


    plt.title('Posterior predictive regression lines')
    plt.legend(loc=0)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.ylim(0.8,1.2)

# Dates Input

### 24 months - Train, 3 months Test , 3 months - Validation

In [ ]:
modelling_start_date = '2020-01-01'
modelling_end_date = '2021-05-31'
modelling_train_start_date = modelling_start_date
modelling_train_end_date = modelling_end_date
modelling_test_start_date = '2021-05-30'
modelling_test_end_date = '2021-05-31'
bayesian_start_date = '2020-10-01'
bayesian_end_date = '2020-03-31'
validation_start_date = '2021-04-01'
validation_end_date = '2021-05-31'

# Modelling

## Input Data

In [ ]:
media_dictionary = {'IMP' : 'Impression',
                    'SPEND' : 'Spend',
                    'VIEW_IMP' : 'Viewable Impression',
                    'CLK' : 'Click'}
# User Inputs
target_var = 'O_UNIT'

# standard names for folders are: new_customer_online, new_customer, sales_unit_online, sales_unit. 
# however the user can choose any folder name
# the folder would be created in models/manual_models
folder_name = 'sales_units'

# standard names for folders are mix_media_metrics, impression, click, spend, viewable_impression
sub_folder = 'mix_media_metrics'

# input files
# input_data_path = "../../data/processed_input_data/modeling_stack_add_competitor_appended_final_DROZ_updated.csv"
input_data_path = "../../data/processed_input_data/02_modelling_stack_data_seasonality.csv"

# Loading data dictionary
data_dict = pd.read_excel("../../data/input_data/WMG WAMM Project Data Dictionary_v12.xlsx")

# loading media hierarchy mapping
media_hier_click = pd.read_csv("../../data/input_data/media_hierarchy_CLK.csv")
media_hier_imp = pd.read_csv("../../data/input_data/media_hierarchy_IMP.csv")
media_hier_view_imp = pd.read_csv("../../data/input_data/media_hierarchy_VIEW_IMP.csv")
media_hier_spend = pd.read_csv("../../data/input_data/media_hierarchy_SPEND.csv")

media_hier = pd.concat([media_hier_click, media_hier_imp, media_hier_view_imp, media_hier_spend], axis= 0)



In [ ]:
# Folder to save the modelling outputs
#folder_path = "C:\\Users\\rohan.garg\\Documents\\Walmart 2021\\Bayesian Update\\Data and Code\\Folder Structure_L'Oreal\\data\\output_data\\bayesian_output\\"

folder_path = "../../data/output_data/bayesian_output/"

In [ ]:
#loading data file
df = pd.read_csv(input_data_path, thousands= ",", na_values = " -   ")
df['index'] = pd.to_datetime(df['index']) # Formatting the date
df.set_index("index", inplace = True, drop= False) 
df['M_SEARCH_MAN_CLK']=df['M_SBA_CLK']+df['M_SP_KWB_CLK']
df['M_SEARCH_AUTO_CLK']=df['M_SP_AB_CLK']

In [ ]:
#Put dummy spends for national media to not break down in built in ROAS calculations
#df['M_NATIONAL_SEARCH_ECOMM_SPEND']=1000
df['M_NATIONAL_TV_SPEND']=1000 
df['M_SEARCH_MAN_SPEND']=df['M_SBA_SPEND']+df['M_SP_KWB_SPEND']
df['M_SEARCH_AUTO_SPEND']=df['M_SP_AB_SPEND']
# df['M_NATIONAL_AUDIO_SPEND']=1000
# df['M_NATIONAL_DISPLAY_ECOMM_SPEND']=1000
# df['M_NATIONAL_SEARCH_SPEND']=1000
# df['M_NATIONAL_DISPLAY_SPEND']=1000
# df['M_NATIONAL_VIDEO_SPEND']=1000

In [ ]:
df.shape

## Feature Engineering

In [ ]:
# df['C_NEUTRO_SPEND']=df['C_NEUTRO_SPEND'].fillna(0)
# df['C_OLAY_SPEND']=df['C_OLAY_SPEND'].fillna(0)
# df['C_COMPETITOR_PRICE']=(df['C_OLAY_PRICE']+df['C_NEUTRO_PRICE'])/2
# df['C_COMPETITOR_SPEND']=(df['C_OLAY_SPEND']+df['C_NEUTRO_SPEND'])/2

In [ ]:
# #proxy spend on ROS
# df.loc['2018-10-01':'2019-12-31','RATE_ON_DIS_ROS']=2.8/1000
# df.loc['2020-01-01':'2021-01-31','RATE_ON_DIS_ROS']=5.6/1000

# df['M_ON_DIS_ROS_SPEND']=df['M_ON_DIS_ROS_IMP']*df['RATE_ON_DIS_ROS']

# df['M_ON_DIS_ROS_HPLO_IMP'] = df['M_ON_DIS_ROS_IMP'] + df['M_ON_DIS_HPLO_IMP'] 

# df['M_ON_DIS_ROS_HPLO_SPEND'] = df['M_ON_DIS_ROS_SPEND'] +df['M_ON_DIS_HPLO_SPEND']

In [ ]:
# #df['D_1H']=0
# #df.loc['2020-01-01':'2020-06-30','D_1H']=1
# #df.loc['2019-01-01':'2019-06-30','D_1H']=1
# df['D_2H']=0
# #df.loc['2020-07-01':'2020-12-31','D_2H']=1
# df.loc['2019-07-01':'2019-12-31','D_2H']=1
# df['D_Q2']=0
# df.loc['2019-05-01':'2019-07-30','D_Q2']=1
# df.loc['2020-05-01':'2020-07-30','D_Q2']=1
# df['D_EVENT_COVID_DATE1']=0
# df.loc['2020-03-15':'2020-04-15','D_EVENT_COVID_DATE1']=1
# df['D_EVENT_COVID_DATE2']=0
# df.loc['2020-04-16':'2020-12-31','D_EVENT_COVID_DATE2']=1

In [ ]:
# df['D_STIMULUS']=0
# df.loc['2020-04-15','D_STIMULUS']=1

# df['D_STORE_PROMO']=0
# df.loc['2018-10-01':'2018-10-31','D_STORE_PROMO']=500

# df.loc['2019-01-01':'2019-02-28','D_STORE_PROMO']=905
# df.loc['2019-04-01':'2019-05-31','D_STORE_PROMO']=1000
# df.loc['2019-08-01':'2019-08-31','D_STORE_PROMO']=2000
# df.loc['2019-09-01':'2019-09-30','D_STORE_PROMO']=2131
# df.loc['2019-11-01':'2019-11-30','D_STORE_PROMO']=750
# df.loc['2019-12-01':'2019-12-31','D_STORE_PROMO']=1874

# df.loc['2020-01-01':'2020-01-31','D_STORE_PROMO']=971
# df.loc['2020-04-01':'2020-04-30','D_STORE_PROMO']=1734
# df.loc['2020-06-01':'2020-06-30','D_STORE_PROMO']=187
# df.loc['2020-08-01':'2020-08-31','D_STORE_PROMO']=782
# df.loc['2020-11-01':'2020-11-30','D_STORE_PROMO']=1622

In [ ]:
# df['O_UNIT_ROLLING_AVG_60D']=df.O_UNIT.rolling(60,min_periods=1).mean()
# df['O_UNIT_ROLLING_AVG_50D']=df.O_UNIT.rolling(50,min_periods=1).mean()
# df['O_UNIT_ROLLING_AVG_42D']=df.O_UNIT.rolling(42,min_periods=1).mean()
# df['O_UNIT_ROLLING_AVG_30D']=df.O_UNIT.rolling(30,min_periods=1).mean()

In [ ]:
cols = df.columns

sarmax = [c for c in cols if ("_SAR" in c)]
sarmax

## Modelling start

In [ ]:
# Creating the modelling object
model_obj = MMM_Modelling(df, target_var, data_dict= data_dict,
                          output_path= folder_path, media_hier= media_hier,
                          apply_adstock = True,
                          comment= 'Media Mix Model')

### Train-Test Split

In [ ]:
# Creating train test split
model_obj.create_train_test_split(modelling_train_start_date, modelling_train_end_date, modelling_test_start_date, modelling_test_end_date)

In [ ]:
# Variable selection
dummy_variables = [#'D_EVENT_PRE_VALENTINES',
#                     'D_HOL_POST_CHRISTMAS',
                    #'D_EVENT_PRE_EASTER',
                     #'D_EVENT_PRE_CHRISTMAS',
#                      'D_EVENT_COVID_DATE1',
#                      'D_EVENT_COVID_DATE2',
                     #'D_EVENT_HOLIDAYS',
#                      'D_EVENT_DR_OZ_SHOW',
                     #'D_EVENT_TALK_TV',
                    #'D_MOD_SKIN',
                    'D_DAY_SUNDAY',
                   'D_DAY_SATURDAY', 
                    'D_DAY_FRIDAY',
                    'D_DAY_MONDAY',
                    #'D_DAY_TUESDAY',
#                     'D_DAY_WEDNESDAY',
                    'D_DAY_THURSDAY',
                    #'D_HOL_JULY_4TH',
                    'D_HOL_CHRISTMAS',
#     'D_HOL_THANKSGIVING'
#                     ,'D_STIMULUS'
                    #,'D_STORE_PROMO' ##negative significant
                    #'D_Q2'
                    #,'D_2H'
                  ]

dummy_variables = [#'D_EVENT_PRE_VALENTINES',
#                     'D_HOL_POST_CHRISTMAS',
                    #'D_EVENT_PRE_EASTER',
                     #'D_EVENT_PRE_CHRISTMAS',
#                      'D_EVENT_COVID_DATE1',
#                      'D_EVENT_COVID_DATE2',
                     #'D_EVENT_HOLIDAYS',
#                      'D_EVENT_DR_OZ_SHOW',
                     #'D_EVENT_TALK_TV',
                    #'D_MOD_SKIN',
                    'D_DAY_SUNDAY',
                   'D_DAY_SATURDAY', 
                    'D_DAY_FRIDAY',
                    'D_DAY_MONDAY',
                    #'D_DAY_TUESDAY',
#                     'D_DAY_WEDNESDAY',
                    'D_DAY_THURSDAY',
                    #'D_HOL_JULY_4TH',
                    'D_HOL_CHRISTMAS',
                    'D_HOL_THANKSGIVING'
#                     ,'D_STIMULUS'
                    #,'D_STORE_PROMO' ##negative significant
                    #'D_Q2'
                    #,'D_2H'
                  ]

media_variables=   [    
'M_ON_DIS_AT_CT_B1_IMP_13_8_4.8_4.7_0',
'M_ON_DIS_AT_CT_B2_IMP_0_2_4.3_20.9_0',
'M_ON_DIS_HPLO_IMP_4_14_4.47_0.15_0',
'M_ON_DIS_KW_IMP_2_2_1.15_4.1_0',
'M_SP_AB_B1_CLK_1_0_3.12_8.03_0',
'M_SP_AB_B2_CLK_0_0_4.68_5.22_0',
'M_SBA_KWB_CLK_2_0_4.0_0.3_0',
'M_OFF_DIS_SMEDIA_IMP_0_0_10.6_15.46_0'    
]                  

media_variables=   [    
'M_ON_DIS_AT_IMP_14_8_2_4_0',
'M_ON_DIS_CT_ROS_IMP_9_0_6_7_0',
'M_ON_DIS_HPLO_IMP_1_3_10_7_0',
'M_ON_DIS_KW_IMP_3_2_3_5_0',
'M_OFF_DIS_WN_IMP_3_8_5_6_0',
'M_OFF_DIS_PIN_IMP_0_0_9_8_0',
'M_NATIONAL_TV_IMP_7_2_2_2_0',
'M_SEARCH_AUTO_CLK_4_4_2_6_0',
'M_SEARCH_MAN_CLK_4_3_2_4_0'
]

competitor_vars = [] #'C_COMPETITOR_PRICE']#'C_COMPETITOR_PRICE','C_COMPETITOR_SPEND' #index does not really work ,'C_OLAY_PRICE' not significant
#'C_OLAY_SPEND','C_NEUTRO_SPEND','C_OLAY_PRICE',

inventory_vars= [] # ['INVENTORY_OMNI_SELLABLE_ITEMS_TOTAL_QTY' ] #,'INVENTORY_OMNI_SELLABLE_ITEMS_TOTAL_QTY'
            #  ,'BRAND_DISTRIBUTION_ACV_WEIGHTED'  'FACING_CAPACITY_ALL_STORES',,'INVENTORY_INSTOCK_RATE_DOTCOM','INVENTORY_OMNI_SELLABLE_ITEMS_TOTAL_QTY''INVENTORY_INSTOCK_RATE_DOTCOM',

other_variables = [
#     'O_UNIT_SMA_DAY_14','O_UNIT_RESIDUAL'] 
#       'O_UNIT_SARIMAX (4, 1, 2) (1, 0, 1, 7)',
        'O_UNIT_SARIMAX (0, 1, 0) (1, 0, 2, 7)']
#     'O_UNIT_TREND','PRICE''O_UNIT_SEASONALITY',,'PRICE''O_UNIT_RESIDUAL' 'O_UNIT_TREND','O_UNIT_RESIDUAL' 'O_UNIT_ROLLING_AVG'

        

# Selecting all the independent variables

##+inventory_vars#+competitor_vars


dummy_variables = ['D_HOL_CHRISTMAS',
'D_HOL_THANKSGIVING',  
'D_HOL_NEW_YEARS_DAY',    
'D_HOL_EASTER',    
'D_HOL_SUPER_BOWL',  
'D_DAY_SATURDAY',     
'D_EVENT_COVID_LOCKDOWN',   
'D_EVENT_PRE_SUPERBOWL',   
'D_DAY_SUNDAY',     
'D_EVENT_COVID_STOCKUP',     
'D_HOL_LABOR_DAY',  
'D_HOL_MONTHER_DAY', 
'D_HOL_FATHER_DAY', 
'D_EVENT_PRODUCT_LAUNCH',        
'D_EVENT_EXTREME_COLD' ]



media_variables=   ['M_ON_DIS_AT_IMP_14_8_2_4_0',
'M_ON_DIS_CT_ROS_IMP_9_0_6_7_0',
'M_ON_DIS_HPLO_IMP_1_3_10_7_0',
'M_ON_DIS_KW_IMP_3_2_3_5_0',
'M_OFF_DIS_WN_IMP_3_8_5_6_0',
'M_OFF_DIS_PIN_IMP_0_0_9_8_0',
'M_NATIONAL_TV_IMP_7_2_2_2_0',
'M_SEARCH_AUTO_CLK_4_4_2_6_0',
'M_SEARCH_MAN_CLK_4_3_2_4_0'
                   ]
                  
                  


other_variables = ['O_NEW_CUSTOMER_SARIMAX (1, 1, 2) (2, 0, 1, 7)',
                   'INVENTORY_DOTCOM_SELLABLE_ITEMS_TOTAL_QTY',
                  'PRICE']

In [ ]:
all_independent_variables = dummy_variables+ media_variables +other_variables +inventory_vars+competitor_vars #+inventory_vars
# create all the media transformations , 
model_obj.create_missing_media_transformations(media_variables, 
                                               trans_start_date= modelling_train_start_date, trans_end_date= modelling_train_end_date)

## Modelling Results

In [ ]:
# Running the linear model and visualizing the results
# model_type will save the results of multiple model iterations
model_obj.create_model_results(X_vars= all_independent_variables, model_type= 'Manual Model 1')

In [ ]:
model_obj.visualize_model_outputs('Manual Model 1', contribution_graph= False, add_residual= False, separate_validation= True,analyst_review= False, show_yaxis= False,)

## Tweaking S-curves

In [ ]:
import ipywidgets
from ipywidgets import *
#interact,FloatSlider,IntSlider
import statistics
from scipy.stats import norm
import ipysheet

In [ ]:
# Construct 4 sliders and select variables to use them

def build_media_range(media_transformed, sliders = [0, 1, 2, 3]):
    
    lag_lower, lag_upper, lag_step = 0, 18, 1
    hlf_lower, hlf_upper, hlf_step = 0, 18, 1
    infle_lower, infle_upper, infle_step = 0.2, 9, 0.2
    scale_lower, scale_upper, scale_step = 0.2, 9, 0.2
    
    for i, mvar in enumerate(media_transformed):
        pieces = mvar.split("_")
        lag = int(pieces[-5])
        hlf = int(pieces[-4])
        infle = np.round(float(pieces[-3]),2)
        scale = np.round(float(pieces[-2]),2)
            
        orig_var = pieces[:-5]
        orig_var = '_'.join([str(s) for s in orig_var])
    
        med_on = (i in sliders) * 1
        piece_df = pd.DataFrame({'id': [i], 'Variable': [orig_var], 'Transformed': [mvar], 'med_on': [med_on],
                                 'lag': [lag], 'hlf': [hlf], 'infle': [infle], 'scale': [scale],
                                 'lag_lower': [lag_lower], 'hlf_lower': [hlf_lower], 'infle_lower': [infle_lower], 'scale_lower': [scale_lower],
                                 'lag_upper': [lag_upper], 'hlf_upper': [hlf_upper], 'infle_upper': [infle_upper], 'scale_upper': [scale_upper],
                                 'lag_step':  [lag_step], 'hlf_step':  [hlf_step], 'infle_step': [infle_step], 'scale_step': [scale_step],
                                })
        if i==0:
            media_df = piece_df.copy()
        else:
            media_df = pd.concat([media_df, piece_df], axis = 0)
            
    media_df.set_index(keys = ['id'], inplace = True)
    return media_df



In [ ]:
# stats.pearsonr(sales, transformed)


In [ ]:
def tweaking(p1_lg, p1_hf, p1_inf, p1_sc, 
             p2_lg, p2_hf, p2_inf, p2_sc, 
             p3_lg, p3_hf, p3_inf, p3_sc,
             p4_lg, p4_hf, p4_inf, p4_sc,
             ):
    global media_var_df
    
    med_df = media_var_df
    slider_vars=  []
    fixed_vars = []
    k = 0
    max_k = 4
    for mid in (med_df.index):
        o_var = med_df.loc[mid, 'Variable']
        t_var= med_df.loc[mid, 'Transformed']
        if (med_df.loc[mid, 'med_on'] == 1):
            k += 1
            s_var = t_var
            if (k == 1):
                s_var = o_var + '_' + str(p1_lg) + '_' + str(p1_hf) + '_' + str(p1_inf) + '_' + str(p1_sc) + '_0'
            if (k == 2):
                s_var = o_var + '_' + str(p2_lg) + '_' + str(p2_hf) + '_' + str(p2_inf) + '_' + str(p2_sc) + '_0'
            if (k == 3):
                s_var = o_var + '_' + str(p3_lg) + '_' + str(p3_hf) + '_' + str(p3_inf) + '_' + str(p3_sc) + '_0'
            if (k == 4):
                s_var = o_var + '_' + str(p4_lg) + '_' + str(p4_hf) + '_' + str(p4_inf) + '_' + str(p4_sc) + '_0'
            if (k > max_k):
                fixed_vars = fixed_vars + [t_var]
            else:
                slider_vars = slider_vars + [s_var]
        else:
            fixed_vars = fixed_vars + [t_var]
        
    media_variables = slider_vars + fixed_vars

    
    all_independent_variables = dummy_variables+ media_variables + other_variables + inventory_vars + competitor_vars 
# create all the media transformations , 
    model_obj.create_missing_media_transformations(media_variables, 
                                               trans_start_date= modelling_train_start_date, trans_end_date= modelling_test_end_date)
    model_obj.create_model_results(X_vars= all_independent_variables, model_type= 'Manual Model 1')

    coeff_summary = model_obj.model_dump['Manual Model 1']['Coefficient Summary'].copy()
    ddf = coeff_summary[coeff_summary['Variables'].str.startswith("M_")]
    corrl_ddf = pd.DataFrame(ddf['Variables'])
    # get RoAS
    roas_stats = model_obj.calculate_ROAS('Manual Model 1', start_date= modelling_train_start_date, end_date= modelling_train_end_date)
    roas_stats.replace([np.inf, -np.inf, np.nan], 0.0, inplace=True)
    
    ddf['Slide'] = 0
    ddf['RoAS'] = 0.0
    mvar_dict = {}
    for i,mvar in enumerate(ddf['Variables'].to_list()):
        pieces = mvar.split("_")
        orig_var = pieces[:-5]
        orig_var = '_'.join([str(s) for s in orig_var]) 
        orig_var = str(i) + '_crrl'
        mvar_dict[mvar] = orig_var
#         print(orig_var)
#         corrl_ddf[orig_var] = 0.0
        ddf[orig_var] = 0.0
    
    
    d_dict = model_obj.data_dict
    data_dict = d_dict[d_dict['Variable Name'].str.startswith('M_')].copy()
    if(target_var == 'O_NEW_CUSTOMER'):
        roas_tag = 'Value NCoAS ($)'
    else:
        roas_tag = 'Value RoAS ($)'
    
    for m in ddf['Variables'].to_list():
        
        ddf.loc[ddf['Variables'] == m, 'Slide'] = (m in slider_vars)*1
        
        variable_desc = data_dict[ data_dict['Variable Name'].apply(lambda c: m.startswith(c))]['Variable Description'].values[0]
        ddf.loc[ddf['Variables'] == m, 'RoAS'] = roas_stats[roas_stats['Display Level'] == variable_desc][roas_tag].values[0]/100.0
        for med2 in ddf['Variables'].to_list():
            ddf.loc[ddf['Variables'] == m, mvar_dict[med2] ] = stats.pearsonr(model_obj.df[med2], model_obj.df[m])[0]

#     ddf['max_RoAS'] = ddf['RoAS'].max()
        
    total_contribution = roas_stats['Total Contribution'].sum()
    total_val = model_obj.df[(model_obj.df.index >= modelling_train_start_date) & (model_obj.df.index <= modelling_train_end_date)][target_var].sum()
#     ddf['mediaContrbn(%)'] 
    mediaContrbn = 100.0*total_contribution/(0.01 + total_val)

    mdl_stats = model_obj.model_dump['Manual Model 1']['Modelling Stats']
    MAPE = mdl_stats.loc[['MAPE']]['Model Fit'][0]
#     ddf['MAPE'] = MAPE
    
    
    print("MaxRoAS=",round(ddf['RoAS'].max(),2), " TotalMediaContrib(%)=", round(mediaContrbn,3), " MAPE=",round(MAPE,2))
    
    return ddf 

#     return ddf[['Slide','Variables','Coefficient','p-value', 'RoAS', 'max_RoAS', 'MAPE', 'mediaContrbn(%)', 'Trgt_Corrl']]



In [ ]:
# you can modify this media variable list for your needs
media_transformed = [
'M_ON_DIS_AT_CT_B1_IMP_13_8_4.8_4.7_0',
'M_ON_DIS_AT_CT_B2_IMP_0_2_4.3_20.9_0',
'M_ON_DIS_HPLO_IMP_4_14_4.47_0.15_0',
'M_ON_DIS_KW_IMP_2_2_1.15_4.1_0',
'M_SP_AB_B1_CLK_1_0_3.12_8.03_0',
'M_SP_AB_B2_CLK_0_0_4.68_5.22_0',
'M_SBA_KWB_CLK_2_0_4.0_0.3_0',
'M_OFF_DIS_SMEDIA_IMP_0_0_10.6_15.46_0' 
]

# choose which media variable you'd like to use in sliders
# for example: sliders = [0,1,2,3] to slide the first 4 variables
media_var_df = build_media_range(media_transformed, 
                                 sliders = [4,5,6,7])

m_df = media_var_df.copy()
ik = 0
for i in (m_df.index):
    if (m_df.loc[i, 'med_on'] == 1):
        ik += 1
        if(ik==1):
            p1_lg = IntSlider(min=m_df.loc[i, 'lag_lower'],max=m_df.loc[i, 'lag_upper'] ,step=m_df.loc[i, 'lag_step'] , value=m_df.loc[i, 'lag'] )
            p1_hf = IntSlider(min=m_df.loc[i, 'hlf_lower'],max=m_df.loc[i, 'hlf_upper'] ,step=m_df.loc[i, 'hlf_step'] , value=m_df.loc[i, 'hlf'] )
            p1_inf = FloatSlider(min=m_df.loc[i, 'infle_lower'],max=m_df.loc[i, 'infle_upper'],step=m_df.loc[i, 'infle_step'],value=m_df.loc[i, 'infle'])
            p1_sc = FloatSlider(min=m_df.loc[i, 'scale_lower'],max=m_df.loc[i, 'scale_upper'],step=m_df.loc[i, 'scale_step'],value=m_df.loc[i, 'scale'])
        if(ik==2):
            p2_lg = IntSlider(min=m_df.loc[i, 'lag_lower'],max=m_df.loc[i, 'lag_upper'] ,step=m_df.loc[i, 'lag_step'] , value=m_df.loc[i, 'lag'] )
            p2_hf = IntSlider(min=m_df.loc[i, 'hlf_lower'],max=m_df.loc[i, 'hlf_upper'] ,step=m_df.loc[i, 'hlf_step'] , value=m_df.loc[i, 'hlf'] )
            p2_inf = FloatSlider(min=m_df.loc[i, 'infle_lower'],max=m_df.loc[i, 'infle_upper'],step=m_df.loc[i, 'infle_step'],value=m_df.loc[i, 'infle'])
            p2_sc = FloatSlider(min=m_df.loc[i, 'scale_lower'],max=m_df.loc[i, 'scale_upper'],step=m_df.loc[i, 'scale_step'],value=m_df.loc[i, 'scale'])
        if(ik==3):
            p3_lg = IntSlider(min=m_df.loc[i, 'lag_lower'],max=m_df.loc[i, 'lag_upper'] ,step=m_df.loc[i, 'lag_step'] , value=m_df.loc[i, 'lag'] )
            p3_hf = IntSlider(min=m_df.loc[i, 'hlf_lower'],max=m_df.loc[i, 'hlf_upper'] ,step=m_df.loc[i, 'hlf_step'] , value=m_df.loc[i, 'hlf'] )
            p3_inf = FloatSlider(min=m_df.loc[i, 'infle_lower'],max=m_df.loc[i, 'infle_upper'],step=m_df.loc[i, 'infle_step'],value=m_df.loc[i, 'infle'])
            p3_sc = FloatSlider(min=m_df.loc[i, 'scale_lower'],max=m_df.loc[i, 'scale_upper'],step=m_df.loc[i, 'scale_step'],value=m_df.loc[i, 'scale'])
        if(ik==4):
            p4_lg = IntSlider(min=m_df.loc[i, 'lag_lower'],max=m_df.loc[i, 'lag_upper'] ,step=m_df.loc[i, 'lag_step'] , value=m_df.loc[i, 'lag'] )
            p4_hf = IntSlider(min=m_df.loc[i, 'hlf_lower'],max=m_df.loc[i, 'hlf_upper'] ,step=m_df.loc[i, 'hlf_step'] , value=m_df.loc[i, 'hlf'] )
            p4_inf = FloatSlider(min=m_df.loc[i, 'infle_lower'],max=m_df.loc[i, 'infle_upper'],step=m_df.loc[i, 'infle_step'],value=m_df.loc[i, 'infle'])
            p4_sc = FloatSlider(min=m_df.loc[i, 'scale_lower'],max=m_df.loc[i, 'scale_upper'],step=m_df.loc[i, 'scale_step'],value=m_df.loc[i, 'scale'])



In [ ]:

interact(tweaking, 
         p1_lg = p1_lg, p1_hf = p1_hf, p1_inf = p1_inf, p1_sc = p1_sc, 
         p2_lg = p2_lg, p2_hf = p2_hf, p2_inf = p2_inf, p2_sc = p2_sc, 
         p3_lg = p3_lg, p3_hf = p3_hf, p3_inf = p3_inf, p3_sc = p3_sc, 
         p4_lg = p4_lg, p4_hf = p4_hf, p4_inf = p4_inf, p4_sc = p4_sc, 
        )
